# data_parquet

> Convert csv to parquet data in dvc parquet stage

In [ ]:
# | hide
from fastcore.basics import *
from fastcore.test import *
from nbdev.showdoc import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
path_input = "../data/test/csv/trades/ETHUSDT"
path_output = "../data/test/parquet/trades/ETHUSDT"
column_time = "timestamp"
sort = True
rename_time = "ts"
drop = "exchange,symbol,local_timestamp,id"

In [ ]:
import copy
import os
import sys

import dask.dataframe as dd

sys.path.append(os.path.abspath("../rlmm/"))

from rlmm.utils import *

In [ ]:
input_abspath = os.path.abspath(path_input)
output_abspath = os.path.abspath(path_output)

print(f"Input path: {input_abspath}")
print(f"Output path: {output_abspath}")

Input path: /home/mvkvc/dev/rlmm/data/test/csv/trades/ETHUSDT
Output path: /home/mvkvc/dev/rlmm/data/test/parquet/trades/ETHUSDT


In [ ]:
if input_abspath.endswith(".csv"):
    read_path = input_abspath
else:
    read_path = input_abspath + "/*.csv"

print(f"Data read path: {read_path}")

Data read path: /home/mvkvc/dev/rlmm/data/test/csv/trades/ETHUSDT/*.csv


In [ ]:
df = dd.read_csv(read_path)

df.head()

OSError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: /home/mvkvc/dev/rlmm/data/test/csv/trades/ETHUSDT/*.csv resolved to no files

In [ ]:
if drop is not None:
    drop_list = drop.split(",")
    df = df.drop(columns=drop_list)

print(f"DF colums: {df.columns}")

In [ ]:
if column_time not in df.columns:
    raise ValueError("time_column does not exist in dataframe")

In [ ]:
if rename_time is not None:
    df = df.rename(columns={column_time: rename_time})
    column_time = copy.deepcopy(rename_time)

print(df.columns)

In [ ]:
df[column_time] = df[column_time].apply(
    dt_from_ts,
    meta=(column_time, "datetime64[ns]"),
)

df[column_time].head()

In [ ]:
if sort:
    df = df.sort_values(column_time)

df = df.set_index(column_time, sorted=True)

df.head()

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [ ]:
if output_abspath.endswith("*.csv"):
    output_abspath = output_abspath[:-5].strip()

if not os.path.exists(output_abspath):
    os.makedirs(output_abspath)

df.to_parquet(output_abspath, write_index=True, write_metadata_file=True)